In [2]:
import os
# v1 = joint
# v2 = no joint
exp_names = ['JT8A8283c_v1', 'JT8A8283c_v2', 'JT8A8287c_v1', 'JT8A8287c_v2', 'JT8A8287c_v1', 'JT8A8287c_v2',
            'JT8A8293c_v1', 'JT8A8293c_v2', 'JT8A8297c_v1', 'JT8A8297c_v2','JT8A8299c_v1', 'JT8A8299c_v2']

# taking this out for now 'JT8A8290c_v1', 'JT8A8290c_v2', 
ds = 4 # downsample factor, for naming things
grid_size = 30

# No time sorry
v1_l1_out_dir = f'detection_results_patch/v1_L1_{str(grid_size)}x{str(grid_size)}_results'
os.makedirs(v1_l1_out_dir, exist_ok=True)
v1_lpips_out_dir = f'detection_results_patch/v1_LPIPS_{str(grid_size)}x{str(grid_size)}_results'
os.makedirs(v1_lpips_out_dir, exist_ok=True)
v2_l1_out_dir = f'detection_results_patch/v2_L1_{str(grid_size)}x{str(grid_size)}_results'
os.makedirs(v2_l1_out_dir, exist_ok=True)
v2_lpips_out_dir = f'detection_results_patch/v2_LPIPS_{str(grid_size)}x{str(grid_size)}_results'
os.makedirs(v2_lpips_out_dir, exist_ok=True)

## Helpers

In [3]:
from PIL import Image
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from util import renormalize, show, losses
import lpips
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (16, 8)
import cv2
import glob
import os
os.environ['HOME'] = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/jingweim/'


# Set up losses (from Lucy's code)
# http://vision38.csail.mit.edu/data/vision/phillipi/gan-training/totem/resources/totems/notebooks/util/losses.py
lpips_loss = losses.LPIPS_Loss()
patch_loss = losses.Patch_Loss(patch_size=7)
l1_loss = nn.L1Loss(reduction='none')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_label = transforms.Compose([
    transforms.ToTensor()
])

In [4]:
from sklearn import metrics

def get_y_valid():
    '''
        y_valid = for cropping out the totem part of the image
    '''
    source_dir = '../data/source-div%s-crop/' % str(ds)
    path_mask = os.path.join(source_dir, 'totem_mask.png')
    mask = cv2.imread(path_mask)[..., 0]
    y, x = np.where(mask) 
    y_valid = np.min(y)
    return y_valid


def get_patches_torch(patch_size, im, im_label, return_labels=False, manip=False):
    '''
        Sample patches of size (patch_size) with a grid size of (grid_size, defined below)
    '''
    _, im_h, im_w = im.size()
#     grid_size = int(max(im_w, im_h) / float(patch_size) * density)
    patches = torch.empty((grid_size ** 2, 3, patch_size, patch_size), dtype=torch.float32)
    labels = torch.empty((3, grid_size ** 2), dtype=torch.float32)
    idx = 0
    for i in np.linspace(0, im_h - patch_size, grid_size).astype(int):
        for j in np.linspace(0, im_w - patch_size, grid_size).astype(int):
            patches[idx] = im[:, i:i+patch_size, j:j+patch_size]
            if return_labels and manip:
                label = torch.mean(im_label[:, i:i+patch_size, j:j+patch_size]) > 0.1
                labels[:, idx] = label
            idx += 1
            
    if not return_labels:
        return patches
    if manip:
        return patches, labels.view((3, grid_size, grid_size))
    else:
        return patches, torch.zeros((3, grid_size, grid_size), dtype=torch.float32)


def detect(out_dir, manip_fname, loss_fn, A_patches, B_patches, grid_size, image, y_valid):
    # Run loss
    with torch.no_grad():
        out = loss_fn(A_patches, B_patches)
    out_ = torch.mean(out, dim=(1, 2, 3)).view(grid_size, grid_size)
    np.save(os.path.join(out_dir, manip_fname+'_raw.npy'), out_)
    
#     # Normalize and resize output
#     out_normalized = (out_ - torch.min(out_)) / (torch.max(out_) - torch.min(out_))
#     im_w, im_h = image.size
#     out = cv2.resize(out_normalized.numpy(), (im_w, y_valid), cv2.INTER_CUBIC)
#     np.save(os.path.join(out_dir, manip_fname+f'_{str(grid_size)}x{str(grid_size)}.npy'), out_normalized)
#     np.save(os.path.join(out_dir, manip_fname+'_result.npy'), out)
#     plt.imsave(os.path.join(out_dir, manip_fname+'_result.png'), out, cmap='jet')
#     return out


def detect_in_one(exp_name, manip_dir, y_valid):
    '''
        This is the main detection method
    
        manip_dir = preprocessed version of Jacob's manipulation
        y_valid = for cropping out the totem part of the image
        all_paths = append path of manipulation image
        l1_scores = append L1 mAP for each image
        lpips_scores = append LPIPs mAP for each image
        manip_pctg = append percentage of manipulated pixels
    '''
    im_name = exp_name[:8]
    paths = glob.glob(os.path.join(manip_dir, '%s*.png' % im_name))
    manip_paths = sorted([path for path in paths if 'mask' not in path])
    manip_mask_paths = sorted([path for path in paths if 'mask' in path])
    
    
    # Load data from other paths
    source_dir = '../data/source-div4-crop/'
    path_nerf = os.path.join(source_dir, f'{exp_name}_nerf.png')
    path_protect = os.path.join(source_dir, f'{exp_name}_protect_mask.png')
    path_orig = os.path.join(source_dir, f'{im_name}.JPG')

    image_nerf = Image.open(path_nerf)
    image_orig = Image.open(path_orig)
    protect_mask = (cv2.imread(path_protect)[:y_valid, :, 0]/255).astype('uint8')
    protect_filter = np.where(protect_mask == 1)


    # Transform
    w, h = image_nerf.size
    nerf_valid_tensor = transform(image_nerf.crop((0, 0, w, y_valid))).clone()
    orig_valid_tensor = transform(image_orig.crop((0, 0, w, y_valid))).clone()


    # Get patches
    patch_size = 64
#     density = 1.0
    nerf_patches = get_patches_torch(patch_size, nerf_valid_tensor, None, return_labels=False, manip=False)
    orig_patches, orig_labels = get_patches_torch(patch_size, orig_valid_tensor, None,return_labels=True, manip=False)
    

    # Detection
    for (manip_path, manip_mask_path) in zip(manip_paths, manip_mask_paths):
        # Load data
        image_manip = Image.open(manip_path)
        image_label = Image.open(manip_mask_path)

        # Transform
        manip_valid_tensor = transform(image_manip.crop((0, 0, w, y_valid))).clone()
        label = transform_label(image_label.crop((0, 0, w, y_valid))).clone()

        # Get patches
        manip_patches, manip_labels = get_patches_torch(patch_size, manip_valid_tensor, label, return_labels=True, manip=True)
        manip_fname = os.path.basename(manip_path).split('.')[0]
        print(manip_fname)

        # Detection
        if 'v1' in exp_name:
            pred_l1 = detect(v1_l1_out_dir, manip_fname, l1_loss, nerf_patches, manip_patches, grid_size, image_manip, y_valid)
            pred_lpips = detect(v1_lpips_out_dir, manip_fname, lpips_loss, nerf_patches, manip_patches, grid_size, image_manip, y_valid)
        else:
            pred_l1 = detect(v2_l1_out_dir, manip_fname, l1_loss, nerf_patches, manip_patches, grid_size, image_manip, y_valid)
            pred_lpips = detect(v2_lpips_out_dir, manip_fname, lpips_loss, nerf_patches, manip_patches, grid_size, image_manip, y_valid)


## Main

In [4]:
y_valid = get_y_valid()

manip_dir = '/data/vision/phillipi/gan-training/totem/resources/totems/data/manip_patch/'

for exp_name in exp_names:
    print(exp_name)
    detect_in_one(exp_name, manip_dir, y_valid)

JT8A8283c_v1
JT8A8283c_0
JT8A8283c_1
JT8A8283c_2
JT8A8283c_v2
JT8A8283c_0
JT8A8283c_1
JT8A8283c_2
JT8A8287c_v1
JT8A8287c_0
JT8A8287c_1
JT8A8287c_2
JT8A8287c_v2
JT8A8287c_0
JT8A8287c_1
JT8A8287c_2
JT8A8287c_v1
JT8A8287c_0
JT8A8287c_1
JT8A8287c_2
JT8A8287c_v2
JT8A8287c_0
JT8A8287c_1
JT8A8287c_2
JT8A8293c_v1
JT8A8293c_0
JT8A8293c_1
JT8A8293c_2
JT8A8293c_v2
JT8A8293c_0
JT8A8293c_1
JT8A8293c_2
JT8A8297c_v1
JT8A8297c_0
JT8A8297c_1
JT8A8297c_2
JT8A8297c_v2
JT8A8297c_0
JT8A8297c_1
JT8A8297c_2
JT8A8299c_v1
JT8A8299c_0
JT8A8299c_1
JT8A8299c_2
JT8A8299c_v2
JT8A8299c_0
JT8A8299c_1
JT8A8299c_2


In [22]:
def get_scores(folder, fname, version, normalize=False):
    # I can also renormalize here!!!
    
    # crop to y_valid for Jacob's pred matrix
    y_valid = 534 # hardcoded cuz out of time wwww
    path = os.path.join(folder, fname)
    out = np.load(path)[:y_valid]
    
    # load manip mask
    manip_dir = '/data/vision/phillipi/gan-training/totem/resources/totems/data/manip_patch/'
    manip_mask_path = os.path.join(manip_dir, fname[:11]+'mask.png')
#     print(fname, manip_mask_path)
    # Make it binary
    manip_label = (cv2.imread(manip_mask_path)[:y_valid, :, 0] == 255).astype(out.dtype)
    
    # load protect mask
    ds = 4
    exp_name = f'{fname[:8]}c_{version}'
    source_dir = '../data/source-div%s-crop/' % str(ds)
    path_protect = os.path.join(source_dir, f'{exp_name}_protect_mask.png')
    protect_mask = cv2.imread(path_protect)[:y_valid, :, 0]
    protect_filter = np.where(protect_mask == 255)
    
    gt = manip_label[protect_filter]
    valid = np.sum(gt) > 0
    out = out[protect_filter]
    if normalize:
        out = (out-np.min(out))/(np.max(out)-np.min(out))
#     print(np.max(out))
        
    if valid:
        score = metrics.average_precision_score(gt, out)
        pct = np.sum(gt)/len(protect_filter[0]) * 100
    else:
        score = 'invalid'
        pct = 0.0
        
    return score, pct


def get_scores_jacob(folder, fname, version, normalize=False):
    # I can also renormalize here!!!
    
    # crop to y_valid for Jacob's pred matrix
    y_valid = 534 # hardcoded cuz out of time wwww
    path = os.path.join(folder, fname)
    out = np.load(path)[:y_valid]
    
    # load manip mask
    manip_dir = '/data/vision/phillipi/gan-training/totem/resources/totems/data/manip_patch/'
    manip_mask_path = os.path.join(manip_dir, fname[:11]+'mask.png')
#     print(fname, manip_mask_path)
    # Make it binary
    manip_label = (cv2.imread(manip_mask_path)[:y_valid, :, 0] == 255).astype(out.dtype)
    
    gt = manip_label.ravel()
    out = out.ravel()
    print('GT stats: ', gt.min(), gt.max())
    print('Estimate stats: ', out.min(), out.max())
    if normalize:
        out = (out-np.min(out))/(np.max(out)-np.min(out))
#     print(np.max(out))
    
    score = metrics.average_precision_score(gt, out)
    pct = np.sum(gt)/len(gt)*100
        
    return score, pct

In [13]:
def average_mAP(scores, pcts, thres=0.0):
    filtered = [score for (score, pct) in zip(scores, pcts) if pct > thres]
    return thres, len(filtered), np.mean(filtered)

In [23]:
# Can add jacob's folder here too
version = 'v2' # 'v2', no joint
root = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_patch/'
lpips_folder = os.path.join(root, f'{version}_LPIPS_30x30_results')
l1_folder = os.path.join(root, f'{version}_L1_30x30_results')
jacob_folder = os.path.join(root, 'jacob_30x30_results')

# folder = lpips_folder
# folder = l1_folder
folder = jacob_folder
fnames = os.listdir(folder)
fnames = sorted([fname for fname in fnames if 'result.npy' in fname])

normalize = False
scores = []
pcts = []
for fname in fnames:
    if 'jacob' in folder:
        score, pct = get_scores_jacob(folder, fname, version, normalize=normalize)
    else:
        score, pct = get_scores(folder, fname, version, normalize=normalize)
    scores.append(score)
    pcts.append(pct)
#     break

GT stats:  0.0 1.0
Estimate stats:  0.02093154769389627 0.4239447373317722
GT stats:  0.0 1.0
Estimate stats:  0.02133570930369011 0.38113688692765724
GT stats:  0.0 1.0
Estimate stats:  0.020734465643552413 0.3470961099232144
GT stats:  0.0 1.0
Estimate stats:  0.010254650544206267 0.542271886812052
GT stats:  0.0 1.0
Estimate stats:  0.01947193648198342 0.4937618758594218
GT stats:  0.0 1.0
Estimate stats:  0.017670674062506886 0.6620452603288853
GT stats:  0.0 1.0
Estimate stats:  0.014334007690854689 0.3859785610287073
GT stats:  0.0 1.0
Estimate stats:  0.01604063902431152 0.3222828871956728
GT stats:  0.0 1.0
Estimate stats:  0.01447527181357311 0.8055074241831773
GT stats:  0.0 1.0
Estimate stats:  0.017512893162627163 0.5072364191998318
GT stats:  0.0 1.0
Estimate stats:  0.017407380296719843 0.4270921642608708
GT stats:  0.0 1.0
Estimate stats:  0.01750079104852209 0.41462301999989604
GT stats:  0.0 1.0
Estimate stats:  0.012915101545471486 0.4382793109172328
GT stats:  0.0 1.

In [21]:
# LPIPS, v1
print(average_mAP(scores, pcts))

(0.0, 15, 0.9458435850099212)


In [23]:
# LPIPS, v2
print(average_mAP(scores, pcts))

(0.0, 15, 0.9543758846844045)


In [26]:
# L1, v1
print(average_mAP(scores, pcts))

(0.0, 15, 0.9606300921019216)


In [29]:
# L1, v2
print(average_mAP(scores, pcts))

(0.0, 15, 0.9442340327567703)


In [10]:
# Jacob
print(average_mAP(scores, pcts))

[0.6390916481181224, 0.6676531160277086, 0.6774641564851147, 0.9150047295993169, 0.6690342409767661, 0.9368103066101485, 0.7465563794366582, 0.2514541386798701, 0.9705580497697417, 0.8174953259227333, 0.5840206962868228, 0.9391945067290588, 0.9429391214072151, 0.9210526981079301, 0.9163734990001847, 0.8147731659838913, 0.7910579063563843, 0.912486809287005, 0.9054628127640303, 0.8943676470970949, 0.8995179990311791]
(0.0, 21, 0.8005889977941417)


## Human CAF only 

In [69]:
# Can add jacob's folder here too
version = 'v2' # 'v2', no joint
root = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_CAF/'
lpips_folder = os.path.join(root, f'{version}_LPIPS_30x30_results')
l1_folder = os.path.join(root, f'{version}_L1_30x30_results')
jacob_folder = os.path.join(root, 'jacob_30x30_results')

# folder = lpips_folder
# folder = l1_folder
folder = jacob_folder
fnames = os.listdir(folder)
fnames = sorted([fname for fname in fnames if 'result.npy' in fname])

normalize = False
scores = []
pcts = []
for fname in fnames:
    if 'jacob' in folder:
        score, pct = get_scores_jacob(folder, fname, version, normalize=normalize)
    else:
        score, pct = get_scores(folder, fname, version, normalize=normalize)
    scores.append(score)
    pcts.append(pct)
#     break

In [62]:
# LPIPS, v1
print(average_mAP(scores, pcts))

(0.0, 7, 0.6663276625484571)


In [68]:
# LPIPS, v2
print(average_mAP(scores, pcts))

(0.0, 7, 0.4892581142591263)


In [64]:
# L1, v1
print(average_mAP(scores, pcts))

(0.0, 7, 0.5542600448963481)


In [66]:
# L1, v2
print(average_mAP(scores, pcts))

(0.0, 7, 0.4851470247974229)


In [70]:
# Jacob
print(average_mAP(scores, pcts))

(0.0, 7, 0.036509224386516444)


## Use all manipulations

In [51]:
# Can add jacob's folder here too
version = 'v2' # 'v2', no joint
root = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results/'
lpips_folder = os.path.join(root, f'{version}_LPIPS_30x30_results')
l1_folder = os.path.join(root, f'{version}_L1_30x30_results')
jacob_folder = os.path.join(root, 'jacob_30x30_results')

# folder = jacob_folder
folder = lpips_folder
fnames = os.listdir(folder)
fnames = sorted([fname for fname in fnames if 'result.npy' in fname])

normalize = False
scores = []
pcts = []
for fname in fnames:
    if 'jacob' in folder:
        score, pct = get_scores_jacob(folder, fname, version, normalize=normalize)
    else:
        score, pct = get_scores(folder, fname, version, normalize=normalize)
    scores.append(score)
    pcts.append(pct)
#     break

0.7725146
0.6694083
0.6694083
0.77363425
0.6694083
0.76344854
0.6531406
0.65340996
0.6531406
0.7415296
0.6531406
0.9781784
0.78097236
0.99702936
0.78097236
0.8495859
0.87052894
0.8496084
0.7109546
0.6223849
0.6223514
0.75098544
0.7236698
0.79600424
0.7923978


In [27]:
# L1, v1
print(average_mAP(scores, pcts, thres=0.0))
print(average_mAP(scores, pcts, thres=1.0))
print(average_mAP(scores, pcts, thres=2.0))
print(average_mAP(scores, pcts, thres=3.0))
print(average_mAP(scores, pcts, thres=5.0))
print(average_mAP(scores, pcts, thres=10.0))


(0.0, 23, 0.25119826018633495)
(1.0, 18, 0.23120224469621042)
(2.0, 10, 0.39647633595262505)
(3.0, 8, 0.4900794105280558)
(5.0, 7, 0.5542600448963481)
(10.0, 2, 0.2137785960080709)


In [31]:
# L1, v2
print(average_mAP(scores, pcts, thres=0.0))
print(average_mAP(scores, pcts, thres=1.0))
print(average_mAP(scores, pcts, thres=2.0))
print(average_mAP(scores, pcts, thres=3.0))
print(average_mAP(scores, pcts, thres=5.0))
print(average_mAP(scores, pcts, thres=10.0))


(0.0, 21, 0.17552704005092737)
(1.0, 16, 0.22880193687103967)
(2.0, 14, 0.25538954860985)
(3.0, 10, 0.3504585852138792)
(5.0, 7, 0.4851470247974229)
(10.0, 6, 0.5498798846052378)


In [29]:
# LPIPS, v1
print(average_mAP(scores, pcts, thres=0.0))
print(average_mAP(scores, pcts, thres=1.0))
print(average_mAP(scores, pcts, thres=2.0))
print(average_mAP(scores, pcts, thres=3.0))
print(average_mAP(scores, pcts, thres=5.0))
print(average_mAP(scores, pcts, thres=10.0))


(0.0, 23, 0.34991703395828383)
(1.0, 18, 0.36687235886173725)
(2.0, 10, 0.5440345043515888)
(3.0, 8, 0.6475023907353621)
(5.0, 7, 0.6663276625484571)
(10.0, 2, 0.6970371696314156)


In [44]:
# LPIPS, v2
print(average_mAP(scores, pcts, thres=0.0))
print(average_mAP(scores, pcts, thres=1.0))
print(average_mAP(scores, pcts, thres=2.0))
print(average_mAP(scores, pcts, thres=3.0))
print(average_mAP(scores, pcts, thres=5.0))
print(average_mAP(scores, pcts, thres=10.0))


(0.0, 21, 0.2139866445842036)
(1.0, 16, 0.2765875907773816)
(2.0, 14, 0.31273856334396893)
(3.0, 10, 0.4059557187557862)
(5.0, 7, 0.4892581142591263)
(10.0, 6, 0.5088670302509949)


In [47]:
# Jacob, with normalize
print(average_mAP(scores, pcts, thres=0.0))
print(average_mAP(scores, pcts, thres=1.0))
print(average_mAP(scores, pcts, thres=2.0))
print(average_mAP(scores, pcts, thres=3.0))
print(average_mAP(scores, pcts, thres=5.0))


(0.0, 25, 0.02288460517930116)
(1.0, 12, 0.03173415695148323)
(2.0, 8, 0.042437576790399445)
(3.0, 7, 0.044713810364436606)
(5.0, 2, 0.039190868617917075)


In [50]:
# Jacob, without normalize
print(average_mAP(scores, pcts, thres=0.0))
print(average_mAP(scores, pcts, thres=1.0))
print(average_mAP(scores, pcts, thres=2.0))
print(average_mAP(scores, pcts, thres=3.0))
print(average_mAP(scores, pcts, thres=5.0))


(0.0, 25, 0.02288460517930116)
(1.0, 12, 0.03173415695148323)
(2.0, 8, 0.042437576790399445)
(3.0, 7, 0.044713810364436606)
(5.0, 2, 0.039190868617917075)


In [82]:
from datetime import datetime

version = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
data = {'paths': all_paths, 'lpips_mAPs': lpips_scores, 'l1_mAPs': l1_scores, 'manip_pct': manip_pctg, 
        'README': stats}

np.save(f"detect_results_{version}.npy", data)

In [85]:
def average_mAP(path, thres=0.0):
    data = np.load(path, allow_pickle=True).item()
    lpips_scores = data['lpips_mAPs']
    l1_scores = data['l1_mAPs']
    manip_pcts = data['manip_pct']
    manip_paths = data['paths']
    
    lpips_scores_filter = []
    l1_scores_filter = []
    manip_paths_filter = []
    for (lpips_score, l1_score, manip_pct, manip_path) in zip(lpips_scores, l1_scores, manip_pcts, manip_paths):
        if manip_pct > thres:
            lpips_scores_filter.append(lpips_score)
            l1_scores_filter.append(l1_score)
            manip_paths_filter.append(manip_path)
    return thres, np.mean(lpips_scores_filter), np.mean(l1_scores_filter), len(manip_paths_filter)

In [86]:
# Compute average mAP for different manipulation size (% manipulated pixels in protexted area)

print( '% threshold, average lpips mAP, average l1 mAP, # images')
print(average_mAP(f"detect_results_{version}.npy", thres=0.0))
print(average_mAP(f"detect_results_{version}.npy", thres=1.0))
print(average_mAP(f"detect_results_{version}.npy", thres=2.0))
print(average_mAP(f"detect_results_{version}.npy", thres=3.0))


(0.0, 0.26218840399472165, 0.19844168946764051, 44)
(1.0, 0.30831585674703205, 0.2264995113783314, 34)
(2.0, 0.4118813702157362, 0.3084099307261024, 24)
(3.0, 0.5214328705220808, 0.4051904764228639, 18)
